# 03 Statical analysis

•	Kiểm định giả thuyết ANOVA, T-test, chi-square.

•	Phân tích sự khác biệt giữa các thành phố, các năm.

•	Báo cáo kết quả thống kê.

# 1. Import libraries

In [1]:
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# 2. Read CSV

In [7]:
df = pd.read_csv(r"D:\weather-tourism-project\data\processed_weather_data.csv",  sep=",", header=0)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df_summer = df[df['month'].between(4, 8)]

# Phân tích thống kê mô tả theo thành phố và nhãn

In [4]:
summary = df.groupby(['city'])[['temp', 'humidity', 'rain_amount', 'wind', 'cloud', 'precip']].describe()
print(summary)

              temp                                                     \
             count       mean       std   min   25%   50%   75%   max   
city                                                                    
DaNang     12312.0  25.834665  4.439968  14.0  22.7  25.5  28.5  42.0   
HaNoi      12312.0  23.732545  5.607076   7.2  19.6  24.7  27.7  41.1   
HoChiMinh  12312.0  27.828484  3.402960  18.4  25.2  27.1  30.3  38.8   

          humidity             ...  cloud          precip                      \
             count       mean  ...    75%    max    count      mean       std   
city                           ...                                              
DaNang     12312.0  82.290530  ...  100.0  100.0  12312.0  0.280011  1.102613   
HaNoi      12312.0  78.492528  ...  100.0  100.0  12312.0  0.257123  1.165626   
HoChiMinh  12312.0  76.348603  ...  100.0  100.0  12312.0  0.211550  0.969225   

                                     
           min  25%  50%  75%   max 

# Anova - So sánh comfort index giữa các thành phố

In [11]:
# ANOVA - So sánh comfort index giữa các thành phố
anova_result = stats.f_oneway(
    df_summer[df_summer['city'] == 'HaNoi']['comfort_index'],
    df_summer[df_summer['city'] == 'DaNang']['comfort_index'],
    df_summer[df_summer['city'] == 'HoChiMinh']['comfort_index']
)
print("ANOVA p-value:", anova_result.pvalue)


ANOVA p-value: 2.640998495222785e-32


# T-test - So sánh comfort index giữa hai năm (nếu có)

In [14]:
df_summer.loc[:, 'year'] = pd.to_datetime(df_summer['date']).dt.year
if df_summer['year'].nunique() >= 2:
    y2024 = df_summer[df_summer['year'] == 2024]['comfort_index']
    y2025 = df_summer[df_summer['year'] == 2025]['comfort_index']
    t_stat, p_val = stats.ttest_ind(y2024, y2025, equal_var=False)
    print("T-test p-value:", p_val)

T-test p-value: 0.3992289669435741


# Kiểm định chi-square giữa thành phố và nhãn

In [13]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(df_summer['city'], df_summer['label'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-square p-value:", p)


Chi-square p-value: 4.9530096579621095e-26
